# A Text Retrieval/Matching System To Match Similar Questions

### Import all libraries to be used

In [225]:
import pandas as pd
import re
import math
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem.snowball import SnowballStemmer
from collections import defaultdict
from collections import Counter
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import timeit

stopwords = stopwords.words('english')

### Define functions for text and query preprocessing

In [226]:
#Transform text/query to lowercases, removing punctuations and numbers and tokenize the text/query
def text_cleaning(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.split()
    return text

#Take a dataframe object and a column as arguments and clean all the texts under that column and remove stopwords
def text_preprocessing(df, col_name):
    df[col_name] = df[col_name].apply(text_cleaning)
    df[col_name] = df.apply(lambda x: [i for i in x[col_name] if not i in stopwords], axis = 1)
    return df

#A POS tag helper which refines the results from the pos_tag package from the nltk library
def tag_helper(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

#Perform POS tagging for a question
def mod_pos(sent):
    sent = pos_tag(sent)
    mod_sent = list(map(lambda x: (x[0], tag_helper(x[1])), sent))
    return mod_sent

#Take a dataframe object and a column as arguments and lemmatize all texts with POS tagging under that column
def text_lemmatize(df, col_name):
    df = text_preprocessing(df, col_name)
    lemmatizer = WordNetLemmatizer()
    df[col_name] = df.apply(lambda x: [lemmatizer.lemmatize(word, tag) if tag else word for word, tag in mod_pos(x[col_name])], axis = 1)
    return df

#Take a query as argument and clean the texts and remove stopwords
def query_preprocessing(query):
    query = text_cleaning(query)
    query = [i for i in query if not i in stopwords]
    return query

#Take a query as argument and lemmatize all texts with POS tagging
def query_lemmatize(query):
    query = query_preprocessing(query)
    lemmatizer = WordNetLemmatizer()
    query = [lemmatizer.lemmatize(word, tag) if tag else word for word, tag in mod_pos(query)]
    return query

### Define functions for calculating similarity with TF/IDF

In [227]:
#Find term frequency for a specific term in a question
def tf(term, sent):
    term_tf = 0
    term_count = sent.count(term)
    if term_count > 0:
        term_tf = math.log(1 + term_count, 10)
    return term_tf

#Find similar questions with TF/IDF scores
def find_tf_idf(question_2, queries_dict):
    
    #declare some dictionaries to store resutls for computations
    all_term_count = defaultdict(int) #this dictionary store the count of number of question that each term occurrs
    inverted_file = defaultdict() #this dictionary is the inverted file with each term as the key, and a dictionary of {question id: TF/IDF} as the value
    tfidf_results = defaultdict() #this dictionary stores the similarities by TF/IDF for all queries
    
    for sent in question_2.values():
        for i in set(sent):
            all_term_count[i] += 1
    N = len(all_term_count)

    #Find all tf/idf for all words in all questions 2 and store to inverted_file
    for key , value in question_2.items():
        for term in value:
            sig = dict()
            term_tf = tf(term, value)
            term_idf = N / all_term_count[term]
            term_tfidf = term_tf * term_idf
            sig.update({key: term_tfidf})
            if term in inverted_file:
                inverted_file[term].update(sig)
            else:
                inverted_file.update({term: sig})
    
    #look up the total tf/idf score of the query question in all questions 2 and store {question id: corresponding tf/idf score} in tfidf_results 
    for key, value in queries_dict.items():
        sent_tfidf = defaultdict()
        for term in value:
            if term in inverted_file:
                sent_tfidf = Counter(sent_tfidf) + Counter(inverted_file[term])
        sent_tfidf = dict(sorted(sent_tfidf.items(), key=lambda item: item[1], reverse=True))
        tfidf_results.update({key: sent_tfidf})
    return tfidf_results

### Define functions for finding similar question with sentence embedding

In [228]:
#calculate cosine similarity of two vectors
def cossim(query, question):
    dot_product = np.dot(query, question)
    norm_1 = np.linalg.norm(query)
    norm_2 = np.linalg.norm(question)
    if norm_1 == 0 or norm_2 == 0:
        return 0
    else:
        return dot_product / (norm_1 * norm_2)

#calculate sentence embedding with average weight
def avgweight_sent_embedding(questions_2, queries_dict):
    avgweight_results = defaultdict() #this dictionary stores the similarities by average weight sentence embedding for all queries
    
    #transform all question 2 to sentence embedding vector by looking up the word in the GloVe model, store as zero vector if the word is not in the model
    question_2_vec = {key: np.mean([model.get(item) if item in model else np.zeros(50) for item in question_2[key]], axis=0) for key, value in question_2.items()}
    
    #preview of embedded vectors of first 20 questions 2 
    i = 1
    for key, value in question_2_vec.items():
        print(value)
        i += 1
        if i > 20:
            break
    i = 0
    
    #find and store the pairwise cosine similarity between all qeueris and all questions 2
    for key, value in queries_dict.items():
        query_sim_dict = defaultdict()
        #transform query to sentence embedding vector by looking up the word in the GloVe model, store as zero vector if the word is not in the model
        query_vec = np.mean([model.get(item) if item in model else np.zeros(50) for item in queries_dict[key]], axis=0)
        #preview of embedded vectors of first 5 queries 
        if i < 5:
            print(query_vec)
            i += 1
        for key2, value2 in question_2_vec.items():
            sim = cossim(query_vec, value2)
            query_sim_dict.update({key2: sim})
        query_sim_dict = dict(sorted(query_sim_dict.items(), key=lambda item: item[1], reverse=True))
        avgweight_results.update({key: query_sim_dict})

    return avgweight_results

#transform a question to a weighted average sentence embedding vector
def idfweight_avg_vec(sent, inverted_idf_file, N):
    #transform each word to word embedding vector by looking up the word in the GloVe model, store as one vector if the word is not in the model
    vec = [model.get(term) if term in model else np.ones(50) for term in sent]
    #look up the term idf for each word in the sentence, if the word does not have a term idf, store as log(N)
    idf = [inverted_idf_file.get(term) if term in inverted_idf_file else math.log(N, 10) for term in sent]
    #find total weight by summing all term idf
    total_weight = sum(idf)
    #calculate the weighted average sentence embedding vector for the question
    sent_vec = sum([v*i for v, i in zip(vec, idf)])/total_weight
    return sent_vec

#calculate sentence embedding with weighted average by multiplying with term idf 
def idfweight_sent_embedding(question_2, queries_dict):
    all_term_count = defaultdict(int)
    
    #build an inverted file for all words in all questions 2
    for sent in question_2.values():
        for i in set(sent):
            all_term_count[i] += 1
    N = len(all_term_count)
    
    #the idf used in this weighted average method is calculated by log(N/term df)
    inverted_idf_file = {term: math.log(N / all_term_count[term], 10) for term, count in all_term_count.items()}
    
    #transform all questions 2 to weighted averaged sentence embedding vectors
    question_2_vec = {key: idfweight_avg_vec(question_2[key], inverted_idf_file, N) for key, value in question_2.items()}
    
    #preview of embedded vectors of first 20 questions 2 
    i = 1
    for key, value in question_2_vec.items():
        print(value)
        i += 1
        if i > 20:
            break
    i = 0
            
    idfweight_results = defaultdict()
    
    #find and store the pairwise cosine similarity between all qeueris and all questions 2
    for key, value in queries_dict.items():
        query_sim_dict = defaultdict()
        query_vec = idfweight_avg_vec(queries_dict[key], inverted_idf_file, N)
        
        #preview of embedded vectors of first 5 queries 
        if i < 5:
            print(query_vec)
            i += 1
        
        for key2, value2 in question_2_vec.items():
            sim = cossim(query_vec, question_2_vec[key2])
            query_sim_dict.update({key2: sim})
        query_sim_dict = dict(sorted(query_sim_dict.items(), key=lambda item: item[1], reverse=True))
        idfweight_results.update({key: query_sim_dict})
        
    return idfweight_results

### Define a function for evaluating the algorithm

In [229]:
#metrics function for evaluating the algorithms
def metrics(results_dict, ground_truth):
    top_5_questions = defaultdict()
    top_2_boo = []
    top_5_boo = []
    
    #store the top 5 similar questions id in a list
    for key, value in results_dict.items():
        i = 0
        top_5 = []
        for qid in value:
            top_5.append(qid)   
            i += 1
            if i > 4:
                break
        top_5_questions.update({key: top_5})
    
    #compare the question id and the question id of the ground truth and see the ground truth is in top 2(5) or not
    for key, value in top_5_questions.items():
        if ground_truth[key] in top_5_questions[key][0:2]:
            top_2_boo.append(True)
            top_5_boo.append(True)
        elif ground_truth[key] in top_5_questions[key][0:5]:
            top_2_boo.append(False)
            top_5_boo.append(True)
        else:
            top_2_boo.append(False)
            top_5_boo.append(False)

    top_2_probi = sum(top_2_boo)/100
    top_5_probi = sum(top_5_boo)/100

    return top_2_probi, top_5_probi

### Define a function for loading the GloVe file

In [230]:
def load_glove_model(file):
    model = {}
    with open(file,'r', encoding="utf8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            model[word] = embedding
    return model

### Functions related to user interface for searching similar questions to user query

In [231]:
#calculate sentence embedding with average weight
def avgweight_user_query(questions_2, query):
    avgweight_results = defaultdict() #this dictionary stores the similarities by average weight sentence embedding for all queries
    
    #transform all question 2 to sentence embedding vector by looking up the word in the GloVe model, store as zero vector if the word is not in the model
    question_2_vec = {key: np.mean([model.get(item) if item in model else np.zeros(50) for item in question_2[key]], axis=0) for key, value in question_2.items()}
    
    #transfor the preprocessed query to sentence embedding vector
    query_vec = np.mean([model.get(item) if item in model else np.zeros(50) for item in query], axis=0)
    
    #find and store the pairwise cosine similarity between all qeueris and all questions 2
    query_results = defaultdict()
    for key, value in question_2_vec.items():
        sim = cossim(query_vec, value)
        query_results.update({key: sim})
    query_results = dict(sorted(query_results.items(), key=lambda item: item[1], reverse=True))
    
    return query_results

#print top 5 most similar questions
def top_5_questions(query_results, ori_question_2):
    query_results
    top_5 = []
    i = 0
    for key, value in query_results.items():
        top_5.append(key)   
        i += 1
        if i > 4:
            break
    print("5 most similar questions:")
    for i in range(5):
        print("------------------------------------------------------------")
        print(f"{i+1}. " + ori_question_2[top_5[i]])
        
def SearchQuestion(query):
    #preprocessing user query
    query = query_lemmatize(query)

    #show top 5 most similar questions
    top_5_questions(avgweight_user_query(question_2, query), ori_question_2)

### Building the system for question matching

#### Start by reading the data file as dataframe object, drop duplicate rows and null rows, and transform the types of values under the columns question id and questions

#### Save the data frame as pickle object

In [232]:
data = pd.read_csv("data.tsv", sep="\t",error_bad_lines=False)
data = data.drop_duplicates(subset=['question1', 'question2'])
data = data.dropna()
data = data.reset_index()
data = data.drop(columns=['index'])
data = data.astype({'qid1': 'int32', 'qid2': 'int32', 'question1': 'string', 'question2': 'string'})
data.to_pickle("all_questions.pkl")

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
C:\Program Files (x86)\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Copy the dataframe and perform text preprocessing, including remove punctuations, numbers and stopwords, lowercasing all letters, tokenization and lemmatization 

#### Save the dataframe as pickle object 

In [233]:
sample = data.copy(deep = True)
sample = text_lemmatize(sample, 'question1')
sample = text_lemmatize(sample, 'question2')
sample.to_pickle("preprocessed.pkl")

#### The below block read the pickle object 

In [234]:
sample = pd.read_pickle("preprocessed.pkl")
data = pd.read_pickle("all_questions.pkl")

#### Define some dictionaries for the algorithms

In [235]:
#stores the question 2 id and original questions of all question 2 as dictionary
ori_question_2 = dict(zip(data.qid2, data.question2))

#stores the question 2 id and preprocessed questions of all question 2 as dictionary
question_2 = dict(zip(sample.qid2, sample.question2))
question_2 = {k: v for k, v in question_2.items() if v} #Further remove all null rows in question 2

#copy queries form the preprocessed dataframe
queries = sample[sample['is_duplicate'] == 1][0:100].copy(deep = True)
queries = queries.reset_index()
queries = queries.drop(columns = ['index'])

#stores queries id and preprocessed queries as dictionary
queries_dict = dict(zip(queries.qid1, queries.question1))

#stores queries id and corresponding ground truth question id as dictionary
ground_truth = dict(zip(queries.qid1, queries.qid2))

### Evaluate three algorithms by the probabilities of the ground truth being ranked in top 2 / 5

#### TF/IDF based

In [236]:
start = timeit.default_timer()

#calculate the probability of the ground truth being in the top 2 or 5 most similar
tf_idf_top_2_probi, tf_idf_top_5_probi = metrics(find_tf_idf(question_2, queries_dict), ground_truth)

In [237]:
print(tf_idf_top_2_probi, tf_idf_top_5_probi)

stop = timeit.default_timer()
print('Time: ', stop - start)  

0.23 0.35
Time:  6.358554899998126


#### Load pretrained GloVe model for sentence embedding model

In [238]:
#laod the GloVe model as a dictionary
model = load_glove_model('glove.6B.50d.txt')

#### Average weighted sentence embedding

In [240]:
start = timeit.default_timer()

#calculate the probability of the ground truth being in the top 2 or 5 most similar
avgweight_top_2_probi, avgweight_top_5_probi = metrics(avgweight_sent_embedding(question_2, queries_dict), ground_truth)

[-0.12911     0.248468    0.129958   -0.36434     0.070104   -0.1778524
  0.186436    0.13053     0.1536316   0.0998572   0.0351416  -0.235438
  0.113802    0.003397    0.1260558   0.102996    0.031054    0.0126578
 -0.178254   -0.38702296  0.0337446   0.110548    0.24546    -0.068448
  0.076268   -0.74305    -0.09686     0.027428   -0.136308   -0.019124
  1.8604      0.302708   -0.138314   -0.485212    0.378984    0.174252
  0.311538    0.43445     0.266024   -0.456542   -0.1284708  -0.203598
  0.09763     0.500448   -0.222418   -0.20583     0.490518    0.381082
  0.208438    0.0678318 ]
[ 0.349345  -0.55255    0.289645   0.10384   -0.149485   0.013528
  0.16477   -0.094675   0.23866   -0.259645   0.319285   0.006904
 -0.26953   -0.0143445 -0.42391    0.408185   0.039643   0.0237745
 -0.0422155 -0.09584   -0.098895  -0.10199   -0.11886   -0.179825
 -0.14502   -0.9998    -0.044797  -0.22443    0.27383    0.2669
  1.8182     0.07165   -0.443585  -0.31221   -0.155905   0.14926
 -0.11482 

[-0.06645    0.644585   0.044554  -0.27556   -0.27594    0.49649
  0.34587    0.454653   0.31185    0.394405   0.069535  -0.092394
 -0.06       0.196425  -0.024185  -0.1918585 -0.035005   0.694735
 -0.579165   0.29998    0.19552    0.543645   0.42286   -0.02839
  0.47564   -0.82548   -0.27159   -0.4447995 -0.264855  -0.198679
  2.1462     0.1311615 -0.3014315  0.591387   0.847415   0.2707
  0.718555   0.56287    0.480704   0.065566   0.02532   -0.19861
 -0.258738   0.658355  -0.49696   -0.218604   0.485645   0.38968
 -0.21189   -0.016369 ]
[-0.0136129   0.41003186 -0.18654614 -0.44983286  0.58566771  0.09707101
 -0.47299714 -0.36177714  0.06095717  0.37900114 -0.23814986  0.47084714
 -0.2919368   0.07176257  0.57260786 -0.03682514  0.13227329 -0.25640286
  0.13095557 -0.21669857  0.107665    0.53817143  0.18287886  0.06108643
  0.60896671 -1.53088571 -0.91847429 -0.37067614  0.35724857 -0.17032857
  3.09582857  0.1423977  -0.45222194 -0.49722286 -0.195407    0.08295571
 -0.10282857  0.

In [241]:
print(avgweight_top_2_probi, avgweight_top_5_probi)

stop = timeit.default_timer()
print('Time: ', stop - start)  

0.4 0.5
Time:  405.42531570000574


#### Weighted with term IDF sentence embedding

In [242]:
start = timeit.default_timer()

idfweight_top_2_probi, idfweight_top_5_probi = metrics(idfweight_sent_embedding(question_2, queries_dict), ground_truth)

[0.54263584 0.76361309 0.70943241 0.37395454 0.67995835 0.49009904
 0.7241952  0.69453419 0.69532611 0.68698935 0.62818441 0.47379579
 0.6707334  0.60516113 0.6915199  0.6869696  0.66502858 0.62550773
 0.49074163 0.33152332 0.62071195 0.67286348 0.77044719 0.55525883
 0.65874442 0.11131941 0.55340583 0.64407735 0.56344138 0.61059932
 1.85741057 0.81553811 0.53753367 0.29749513 0.83963319 0.71804356
 0.77909737 0.88530405 0.76504132 0.30589103 0.52606798 0.48782299
 0.67962786 0.90943654 0.48080611 0.4986744  0.89368005 0.84338339
 0.72366842 0.65921291]
[0.91629302 0.41518182 0.88312249 0.77988545 0.63913293 0.72970627
 0.8137394  0.66958648 0.85479418 0.5779258  0.89959107 0.72602584
 0.57243349 0.71421974 0.48665682 0.94898572 0.74421629 0.73539943
 0.69873405 0.66893918 0.66724176 0.66552212 0.6561488  0.62227541
 0.64161378 0.16668063 0.69729971 0.59749197 0.87433535 0.8704849
 1.73241872 0.76200003 0.47572499 0.54871961 0.63556585 0.80512173
 0.65839351 0.72574914 0.74727332 0.820

[-2.49975961e-01  7.54339966e-01  1.10458146e-03 -4.84559763e-01
 -8.36382902e-01  5.26975752e-01  7.81601526e-01  6.58875281e-01
  5.32395062e-01  6.53011514e-01  1.89429416e-01 -5.33787851e-02
  1.48296838e-01  1.62085198e-01 -2.70774388e-01 -1.42022336e-01
 -1.40855364e-01  9.57303891e-01 -4.96872246e-01  6.16050616e-01
  4.03754249e-01  7.16364696e-01  5.43459748e-01 -1.55734937e-01
  6.36864949e-01 -3.06654097e-01 -8.77029477e-02 -6.48099072e-01
 -4.42360362e-01 -8.39140093e-02  1.35911681e+00  2.15531011e-01
 -4.24100784e-01  8.62982216e-01  1.00561400e+00  3.36741229e-01
  9.52660251e-01  7.10619014e-01  6.82826655e-01  1.28725237e-01
  1.12405642e-01 -4.42606751e-01 -3.38251312e-01  8.15018160e-01
 -6.65868977e-01 -3.31715192e-01  8.36219113e-01  6.76117040e-01
 -3.47650777e-01  3.14448112e-02]
[ 0.06105066  0.32787531 -0.13310218 -0.55344317  0.58811996  0.08622158
 -0.36219299 -0.3398421   0.04166713  0.34791013 -0.22980778  0.42319703
 -0.25306946  0.01908633  0.46137576  0.

In [243]:
print(idfweight_top_2_probi, idfweight_top_5_probi)

stop = timeit.default_timer()
print('Time: ', stop - start) 

0.36 0.51
Time:  382.4497138000006


#### A user interface for finding similar questions with average weighted sentence embedding algorithm and the results

In [244]:
#user interface for showing top 5 most similar questions
SearchQuestion("Why am I tired of doing assignments?")

5 most similar questions:
------------------------------------------------------------
1. How do I get my boyfriend to leave his crappy job?
------------------------------------------------------------
2. I have an extremely stressful job, where my clients get upset with me daily. What can I do to stay calm, even when I am getting yelled at all day?
------------------------------------------------------------
3. What does it mean when a guy is emotionally unavailable?
------------------------------------------------------------
4. Do escorts feel nervous when they know they will have a very good-looking client?
------------------------------------------------------------
5. Are you happy with your job?
